In [ ]:
from bs4 import BeautifulSoup
import requests
import csv

topic = input("Enter name : ")
url = f'https://news.google.com/search?q={topic}&hl=en-IN&gl=IN&ceid=IN%3Aen'
# url = 'https://news.google.com/home?hl=en-IN&gl=IN&ceid=IN:en'
page = requests.get(url)

soup = BeautifulSoup(page.text, 'html.parser')
articles = soup.find_all('a', class_='WwrzSb')
articles_list = []

for article in articles:
    link = 'https://news.google.com' + article['href'][1:]
    try:
        p1 = requests.get(link)
        if p1 is None:
            continue  # Skip this article if unable to retrieve the content

        if p1.status_code == 403:  # Access Denied
            continue

        if p1.status_code == 404:  # Not Found
            continue

        soup1 = BeautifulSoup(p1.text, 'html.parser')
        heading_tag = soup1.find('h1')

        if heading_tag:
            heading = heading_tag.get_text(strip=True)
            merged_text = '\n'.join([p.get_text(strip=True) for p in soup1.find_all('p')])
            articles_list.append((heading, merged_text))
    except requests.exceptions.ConnectionError as e:
        print(f"Connection error occurred: {e}")
        continue  # Continue to the next iteration if connection error occurs

# Save the scraped articles in a CSV file
file_name = f"{topic}_articles.csv"
with open(file_name, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Heading', 'Merged Text'])
    writer.writerows(articles_list)

print("Articles saved in:", file_name)


Enter name : google
Connection error occurred: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Connection error occurred: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Connection error occurred: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Connection error occurred: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Articles saved in: google_articles.csv


In [ ]:
import pandas as pd
df= pd.read_csv(f'{topic}_articles.csv')
df

,Heading,Merged Text
0,Google to destroy browsing data to settle cons...,"Users alleged that Google's analytics, cookies..."
1,Google to finally delete data of users who use...,Copyright © HT Media LimitedAll rights reserve...
2,How to sign out of a single Google account,The TOI Tech Desk is a dedicated team of journ...
3,Secure your home with best Google smart locks:...,Looking to enhance your home's security and co...
4,Delhi HC dismisses Google's appeal against mes...,5 Stories\n7 Stories\n9 Stories\n9 Stories\n8 ...
...,...,...
75,Why the Delhi High Court imposed a ₹1 lakh fin...,The Delhi High Court on Tuesday rejected an ap...
76,"Google Podcasts shuts down tomorrow, April 2",Front page layout\nSite theme\nRon Amadeo-Apr ...
77,Google Must Delete Over 100 Billion Chrome Pri...,Google will delete billions of Chrome incognit...
78,"Google is funding AI tools for nonprofits, inc...","As brands, tech giants, and consumers navigate..."


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
def tokenize_text(text):
    return word_tokenize(text)
df['tokens'] = df['Merged Text'].apply(tokenize_text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    return [word for word in tokens if word.lower() not in stop_words]
df['tokens'] = df['tokens'].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def lower_case(tokens):
    return [token.lower() for token in tokens]
df['tokens'] = df['tokens'].apply(lower_case)
df

,Heading,Merged Text,tokens
0,Google to destroy browsing data to settle cons...,"Users alleged that Google's analytics, cookies...","[users, alleged, google, 's, analytics, ,, coo..."
1,Google to finally delete data of users who use...,Copyright © HT Media LimitedAll rights reserve...,"[copyright, ©, ht, media, limitedall, rights, ..."
2,How to sign out of a single Google account,The TOI Tech Desk is a dedicated team of journ...,"[toi, tech, desk, dedicated, team, journalists..."
3,Secure your home with best Google smart locks:...,Looking to enhance your home's security and co...,"[looking, enhance, home, 's, security, conveni..."
4,Delhi HC dismisses Google's appeal against mes...,5 Stories\n7 Stories\n9 Stories\n9 Stories\n8 ...,"[5, stories, 7, stories, 9, stories, 9, storie..."
...,...,...,...
75,Why the Delhi High Court imposed a ₹1 lakh fin...,The Delhi High Court on Tuesday rejected an ap...,"[delhi, high, court, tuesday, rejected, appeal..."
76,"Google Podcasts shuts down tomorrow, April 2",Front page layout\nSite theme\nRon Amadeo-Apr ...,"[front, page, layout, site, theme, ron, amadeo..."
77,Google Must Delete Over 100 Billion Chrome Pri...,Google will delete billions of Chrome incognit...,"[google, delete, billions, chrome, incognito, ..."
78,"Google is funding AI tools for nonprofits, inc...","As brands, tech giants, and consumers navigate...","[brands, ,, tech, giants, ,, consumers, naviga..."


In [ ]:
def replace_apostrophes(tokens):
    appos = {
        "ain't": "am not",
        "aren't": "are not",
        "can't": "cannot",
        "can't've": "cannot have",
        "cause": "because",
        "could've": "could have",
        "couldn't": "could not",
        "couldn't've": "could not have",
        "didn't": "did not",
        "doesn't": "does not",
        "don't": "do not",
        "hadn't": "had not",
        "hadn't've": "had not have",
        "hasn't": "has not",
        "haven't": "have not",
        "he'd": "he would",
        "he'd've": "he would have",
        "he'll": "he will",
        "he'll've": "he will have",
        "he's": "he is",
        "how'd": "how did",
        "how'd'y": "how do you",
        "how'll": "how will",
        "how's": "how is",
        "i'd": "I would",
        "i'd've": "I would have",
        "i'll": "I will",
        "i'll've": "I will have",
        "i'm": "I am",
        "i've": "I have",
        "isn't": "is not",
        "it'd": "it had",
        "it'd've": "it would have",
        "it'll": "it will",
        "it'll've": "it will have",
        "it's": "it is",
        "let's": "let us",
        "ma'am": "madam",
        "mayn't": "may not",
        "might've": "might have",
        "mightn't": "might not",
        "mightn't've": "might not have",
        "must've": "must have",
        "mustn't": "must not",
        "mustn't've": "must not have",
        "needn't": "need not",
        "needn't've": "need not have",
        "o'clock": "of the clock",
        "oughtn't": "ought not",
        "oughtn't've": "ought not have",
        "shan't": "shall not",
        "sha'n't": "shall not",
        "shan't've": "shall not have",
        "she'd": "she would",
        "she'd've": "she would have",
        "she'll": "she will",
        "she'll've": "she will have",
        "she's": "she is",
        "should've": "should have",
        "shouldn't": "should not",
        "shouldn't've": "should not have",
        "so've": "so have",
        "so's": "so is",
        "that'd": "that would",
        "that'd've": "that would have",
        "that's": "that is",
        "there'd": "there had",
        "there'd've": "there would have",
        "there's": "there is",
        "they'd": "they would",
        "they'd've": "they would have",
        "they'll": "they will",
        "they'll've": "they will have",
        "they're": "they are",
        "they've": "they have",
        "to've": "to have",
        "wasn't": "was not",
        "we'd": "we had",
        "we'd've": "we would have",
        "we'll": "we will",
        "we'll've": "we will have",
        "we're": "we are",
        "we've": "we have",
        "weren't": "were not",
        "what'll": "what will",
        "what'll've": "what will have",
        "what're": "what are",
        "what's": "what is",
        "what've": "what have",
        "when's": "when is",
        "when've": "when have",
        "where'd": "where did",
        "where's": "where is",
        "where've": "where have",
        "who'll": "who will",
        "who'll've": "who will have",
        "who's": "who is",
        "who've": "who have",
        "why's": "why is",
        "why've": "why have",
        "will've": "will have",
        "won't": "will not",
        "won't've": "will not have",
        "would've": "would have",
        "wouldn't": "would not",
        "wouldn't've": "would not have",
        "y'all": "you all",
        "y'alls": "you all's",
        "y'all'd": "you all would",
        "y'all'd've": "you all would have",
        "y'all're": "you all are",
        "y'all've": "you all have",
        "you'd": "you had",
        "you'd've": "you would have",
        "you'll": "you will",
        "you'll've": "you will have",
        "you're": "you are",
        "you've": "you have",
    }
    reformed = [appos[word] if word in appos else word for word in tokens]
    return reformed
df['tokens'] = df['tokens'].apply(replace_apostrophes)
df

,Heading,Merged Text,tokens
0,Google to destroy browsing data to settle cons...,"Users alleged that Google's analytics, cookies...","[users, alleged, google, 's, analytics, ,, coo..."
1,Google to finally delete data of users who use...,Copyright © HT Media LimitedAll rights reserve...,"[copyright, ©, ht, media, limitedall, rights, ..."
2,How to sign out of a single Google account,The TOI Tech Desk is a dedicated team of journ...,"[toi, tech, desk, dedicated, team, journalists..."
3,Secure your home with best Google smart locks:...,Looking to enhance your home's security and co...,"[looking, enhance, home, 's, security, conveni..."
4,Delhi HC dismisses Google's appeal against mes...,5 Stories\n7 Stories\n9 Stories\n9 Stories\n8 ...,"[5, stories, 7, stories, 9, stories, 9, storie..."
...,...,...,...
75,Why the Delhi High Court imposed a ₹1 lakh fin...,The Delhi High Court on Tuesday rejected an ap...,"[delhi, high, court, tuesday, rejected, appeal..."
76,"Google Podcasts shuts down tomorrow, April 2",Front page layout\nSite theme\nRon Amadeo-Apr ...,"[front, page, layout, site, theme, ron, amadeo..."
77,Google Must Delete Over 100 Billion Chrome Pri...,Google will delete billions of Chrome incognit...,"[google, delete, billions, chrome, incognito, ..."
78,"Google is funding AI tools for nonprofits, inc...","As brands, tech giants, and consumers navigate...","[brands, ,, tech, giants, ,, consumers, naviga..."


In [ ]:
def filter_alpha(tokens):
    return [word for word in tokens if word.isalpha()]

# Apply the function to the 'tokens' column
df['tokens'] = df['tokens'].apply(filter_alpha)
df

,Heading,Merged Text,tokens
0,Google to destroy browsing data to settle cons...,"Users alleged that Google's analytics, cookies...","[users, alleged, google, analytics, cookies, a..."
1,Google to finally delete data of users who use...,Copyright © HT Media LimitedAll rights reserve...,"[copyright, ht, media, limitedall, rights, res..."
2,How to sign out of a single Google account,The TOI Tech Desk is a dedicated team of journ...,"[toi, tech, desk, dedicated, team, journalists..."
3,Secure your home with best Google smart locks:...,Looking to enhance your home's security and co...,"[looking, enhance, home, security, convenience..."
4,Delhi HC dismisses Google's appeal against mes...,5 Stories\n7 Stories\n9 Stories\n9 Stories\n8 ...,"[stories, stories, stories, stories, stories, ..."
...,...,...,...
75,Why the Delhi High Court imposed a ₹1 lakh fin...,The Delhi High Court on Tuesday rejected an ap...,"[delhi, high, court, tuesday, rejected, appeal..."
76,"Google Podcasts shuts down tomorrow, April 2",Front page layout\nSite theme\nRon Amadeo-Apr ...,"[front, page, layout, site, theme, ron, pm, ut..."
77,Google Must Delete Over 100 Billion Chrome Pri...,Google will delete billions of Chrome incognit...,"[google, delete, billions, chrome, incognito, ..."
78,"Google is funding AI tools for nonprofits, inc...","As brands, tech giants, and consumers navigate...","[brands, tech, giants, consumers, navigate, bo..."


In [ ]:
from nltk.stem import WordNetLemmatizer

# Download WordNet if not already downloaded
nltk.download('wordnet')

# Initialize WordNet lemmatizer
lemmatizer = WordNetLemmatizer()

# Define the function to lemmatize tokens
def lemmatize_tokens(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

# Apply the function to the 'tokens' column
df['tokens'] = df['tokens'].apply(lemmatize_tokens)
df

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,Heading,Merged Text,tokens
0,Google to destroy browsing data to settle cons...,"Users alleged that Google's analytics, cookies...","[user, alleged, google, analytics, cooky, apps..."
1,Google to finally delete data of users who use...,Copyright © HT Media LimitedAll rights reserve...,"[copyright, ht, medium, limitedall, right, res..."
2,How to sign out of a single Google account,The TOI Tech Desk is a dedicated team of journ...,"[toi, tech, desk, dedicated, team, journalist,..."
3,Secure your home with best Google smart locks:...,Looking to enhance your home's security and co...,"[looking, enhance, home, security, convenience..."
4,Delhi HC dismisses Google's appeal against mes...,5 Stories\n7 Stories\n9 Stories\n9 Stories\n8 ...,"[story, story, story, story, story, story, dow..."
...,...,...,...
75,Why the Delhi High Court imposed a ₹1 lakh fin...,The Delhi High Court on Tuesday rejected an ap...,"[delhi, high, court, tuesday, rejected, appeal..."
76,"Google Podcasts shuts down tomorrow, April 2",Front page layout\nSite theme\nRon Amadeo-Apr ...,"[front, page, layout, site, theme, ron, pm, ut..."
77,Google Must Delete Over 100 Billion Chrome Pri...,Google will delete billions of Chrome incognit...,"[google, delete, billion, chrome, incognito, d..."
78,"Google is funding AI tools for nonprofits, inc...","As brands, tech giants, and consumers navigate...","[brand, tech, giant, consumer, navigate, boom,..."


,text,label
0,Wall St. Bears Claw Back Into the Black (Reute...,2
1,Carlyle Looks Toward Commercial Aerospace (Reu...,2
2,Oil and Economy Cloud Stocks' Outlook (Reuters...,2
3,Iraq Halts Oil Exports from Main Southern Pipe...,2
4,"Oil prices soar to all-time record, posing new...",2
...,...,...
119995,Pakistan's Musharraf Says Won't Quit as Army C...,0
119996,Renteria signing a top-shelf deal Red Sox gene...,1
119997,Saban not going to Dolphins yet The Miami Dolp...,1
119998,Today's NFL games PITTSBURGH at NY GIANTS Time...,1


In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt') # Download 'punkt' if not already downloaded
nltk.download('stopwords')

def preprocessing(file_path):
    # Read CSV file
    df = pd.read_csv(file_path)

    # Function to remove null rows
    def remove_null_rows(df):
        return df.dropna().reset_index(drop=True)

    df = remove_null_rows(df)

    # Tokenization
    def tokenize_text(text):
        return word_tokenize(text)
    df['tokens'] = df['text'].apply(tokenize_text)

    def remove_stopwords(tokens):
        stop_words = set(stopwords.words('english'))
        return [word for word in tokens if word.lower() not in stop_words]
    df['tokens'] = df['tokens'].apply(remove_stopwords)

    # Convert tokens to lowercase
    def lower_case(tokens):
        return [token.lower() for token in tokens]
    df['tokens'] = df['tokens'].apply(lower_case)

    # Replace contractions and apostrophes
    def replace_apostrophes(tokens):
        appos = {
            # dictionary of contractions and their expansions
        }
        reformed = [appos[word] if word in appos else word for word in tokens]
        return reformed
    df['tokens'] = df['tokens'].apply(replace_apostrophes)

    # Filter out non-alphabetic tokens
    def filter_alpha(tokens):
        return [word for word in tokens if word.isalpha()]
    df['tokens'] = df['tokens'].apply(filter_alpha)

    # Lemmatization
    nltk.download('wordnet') # Download WordNet if not already downloaded
    lemmatizer = WordNetLemmatizer() # Initialize WordNet lemmatizer
    def lemmatize_tokens(tokens):
        return [lemmatizer.lemmatize(token) for token in tokens]
    df['tokens'] = df['tokens'].apply(lemmatize_tokens)

    return df


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df = preprocessing('/content/train.csv')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
df['label'] = df['label'].map({0: 0, 1: 1, 2: 1, 3: 1})

In [ ]:
! pip install tensorflow scikit-learn pandas numpy pickle5

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
import pickle5 as pickle

# Assuming 'preprocessed_df' is your preprocessed DataFrame containing the 'tokens' and 'label' columns
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['tokens'])

sequences = tokenizer.texts_to_sequences(df['tokens'])

# Pad sequences
maxlen = 100  # Define the maximum sequence length
padded_sequences = pad_sequences(sequences, maxlen=maxlen, truncating='post')

# Assuming 'preprocessed_df' is your preprocessed DataFrame containing the 'label' column
sentiment_labels = df['label'].values

# Now 'sentiment_labels' contains the sentiment labels from the 'label' column
from sklearn.model_selection import train_test_split

# Split padded_sequences and sentiment_labels into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(padded_sequences, sentiment_labels, test_size=0.2)

# Now x_train, x_test contain the padded sequences, and y_train, y_test contain the sentiment labels


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Define the model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=100, input_length=100))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(x_train, y_train, batch_size=64, epochs=100, validation_split=0.1, callbacks=[early_stopping])

# Evaluate the model on the test set
loss, accuracy = model.evaluate(x_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

Epoch 1/100


InvalidArgumentError: Graph execution error:

Detected at node sequential_5/embedding_5/embedding_lookup defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/u

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Function for preprocessing
def preprocessing(file_path):
    # Read CSV file
    df = pd.read_csv(file_path)

    # Function to remove null rows
    def remove_null_rows(df):
        return df.dropna().reset_index(drop=True)

    df = remove_null_rows(df)

    # Tokenization
    def tokenize_text(text):
        return word_tokenize(text)
    df['tokens'] = df['text'].apply(tokenize_text)

    def remove_stopwords(tokens):
        stop_words = set(stopwords.words('english'))
        return [word for word in tokens if word.lower() not in stop_words]
    df['tokens'] = df['tokens'].apply(remove_stopwords)

    # Convert tokens to lowercase
    def lower_case(tokens):
        return [token.lower() for token in tokens]
    df['tokens'] = df['tokens'].apply(lower_case)

    # Replace contractions and apostrophes
    def replace_apostrophes(tokens):
        appos = {
            # dictionary of contractions and their expansions
        }
        reformed = [appos[word] if word in appos else word for word in tokens]
        return reformed
    df['tokens'] = df['tokens'].apply(replace_apostrophes)

    # Filter out non-alphabetic tokens
    def filter_alpha(tokens):
        return [word for word in tokens if word.isalpha()]
    df['tokens'] = df['tokens'].apply(filter_alpha)

    # Lemmatization
    lemmatizer = WordNetLemmatizer() # Initialize WordNet lemmatizer
    def lemmatize_tokens(tokens):
        return [lemmatizer.lemmatize(token) for token in tokens]
    df['tokens'] = df['tokens'].apply(lemmatize_tokens)

    return df

# Preprocess data
df = preprocessing('/content/train.csv')
df['label'] = df['label'].map({0: 0, 1: 1, 2: 1, 3: 1})

# Initialize Tokenizer with num_words parameter
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df['tokens'])

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(df['tokens'])

# Pad sequences
maxlen = 100  # Define the maximum sequence length
padded_sequences = pad_sequences(sequences, maxlen=maxlen, padding='post', truncating='post')

# Prepare labels
sentiment_labels = df['label'].values

# Split data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(padded_sequences, sentiment_labels, test_size=0.2)

# Define the model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=100, input_length=100))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1, activation='sigmoid'))


# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
history = model.fit(x_train, y_train, batch_size=64, epochs=100, validation_split=0.1, callbacks=[early_stopping])

# Evaluate the model on the test set
loss, accuracy = model.evaluate(x_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Epoch 1/100
1350/1350 [==============================] - 104s 76ms/step - loss: 0.1891 - accuracy: 0.9343 - val_loss: 0.1445 - val_accuracy: 0.9505
Epoch 2/100
1350/1350 [==============================] - 103s 76ms/step - loss: 0.1167 - accuracy: 0.9615 - val_loss: 0.1406 - val_accuracy: 0.9525
Epoch 3/100
1350/1350 [==============================] - 105s 78ms/step - loss: 0.0741 - accuracy: 0.9749 - val_loss: 0.1656 - val_accuracy: 0.9511
Epoch 4/100
1350/1350 [==============================] - 106s 78ms/step - loss: 0.0386 - accuracy: 0.9868 - val_loss: 0.2075 - val_accuracy: 0.9481
Epoch 5/100
750/750 [==============================] - 9s 12ms/step - loss: 0.1442 - accuracy: 0.9533
Test Loss: 0.14417503774166107
Test Accuracy: 0.95333331823349


In [ ]:
df1 = pd.read_csv('/content/Fin_Cleaned.csv')
df1

,Date_published,Headline,Synopsis,Full_text,Final Status
0,2022-06-21,"Banks holding on to subsidy share, say payment...",The companies have written to the National Pay...,ReutersPayments companies and banks are at log...,Negative
1,2022-04-19,Digitally ready Bank of Baroda aims to click o...,"At present, 50% of the bank's retail loans are...",AgenciesThe bank presently has 20 million acti...,Positive
2,2022-05-27,Karnataka attracted investment commitment of R...,Karnataka is at the forefront in attracting in...,PTIKarnataka Chief Minister Basavaraj Bommai.K...,Positive
3,2022-04-06,Splitting of provident fund accounts may be de...,The EPFO is likely to split accounts only at t...,Getty ImagesThe budget for FY22 had imposed in...,Negative
4,2022-06-14,Irdai weighs proposal to privatise Insurance I...,"Set up in 2009 as an advisory body, IIB collec...",AgenciesThere is a view in the insurance indus...,Positive
...,...,...,...,...,...
395,2022-06-10,"Banks take a cue from RBI, hike lending rates",These banks raised their respective external b...,"PTIICICI Bank, Bank of Baroda, Punjab National...",Negative
396,2022-06-29,Sebi issues Rs 27 lakh recovery notice to indi...,"In the event of non-payment, it will recover t...",ReutersThe logo of the Securities and Exchange...,Negative
397,2022-06-06,Apollo Hospital shares drop 0.68% as Sensex ...,"A total of 10,105 shares changed hands on the ...",Getty ImagesShrikant Chouhan of Kotak Securiti...,Negative
398,2022-05-16,SBI at Rs 710? What makes analysts see up to 5...,Calling the stock 'attractively valued' analys...,AgenciesThe PSU bank reported a 41.27 per cent...,Positive


In [ ]:
# Replace values in the 'Final Status' column
df1['Final Status'] = df1['Final Status'].replace({'Negative': 0, 'Positive': 1})

# Print the DataFrame to see the changes
df1


,Date_published,Headline,Synopsis,Full_text,Final Status
0,2022-06-21,"Banks holding on to subsidy share, say payment...",The companies have written to the National Pay...,ReutersPayments companies and banks are at log...,0
1,2022-04-19,Digitally ready Bank of Baroda aims to click o...,"At present, 50% of the bank's retail loans are...",AgenciesThe bank presently has 20 million acti...,1
2,2022-05-27,Karnataka attracted investment commitment of R...,Karnataka is at the forefront in attracting in...,PTIKarnataka Chief Minister Basavaraj Bommai.K...,1
3,2022-04-06,Splitting of provident fund accounts may be de...,The EPFO is likely to split accounts only at t...,Getty ImagesThe budget for FY22 had imposed in...,0
4,2022-06-14,Irdai weighs proposal to privatise Insurance I...,"Set up in 2009 as an advisory body, IIB collec...",AgenciesThere is a view in the insurance indus...,1
...,...,...,...,...,...
395,2022-06-10,"Banks take a cue from RBI, hike lending rates",These banks raised their respective external b...,"PTIICICI Bank, Bank of Baroda, Punjab National...",0
396,2022-06-29,Sebi issues Rs 27 lakh recovery notice to indi...,"In the event of non-payment, it will recover t...",ReutersThe logo of the Securities and Exchange...,0
397,2022-06-06,Apollo Hospital shares drop 0.68% as Sensex ...,"A total of 10,105 shares changed hands on the ...",Getty ImagesShrikant Chouhan of Kotak Securiti...,0
398,2022-05-16,SBI at Rs 710? What makes analysts see up to 5...,Calling the stock 'attractively valued' analys...,AgenciesThe PSU bank reported a 41.27 per cent...,1


In [ ]:
df1.rename(columns={'Headline': 'Headline', 'Full_text': 'text', 'Final Status': 'label'}, inplace=True)

# Save to CSV file
df1.to_csv('full_articles1.csv', index=False)

In [ ]:
df2 = pd.read_csv('/content/full_articles1.csv')
df2

,Date_published,Headline,Synopsis,text,label
0,2022-06-21,"Banks holding on to subsidy share, say payment...",The companies have written to the National Pay...,ReutersPayments companies and banks are at log...,0
1,2022-04-19,Digitally ready Bank of Baroda aims to click o...,"At present, 50% of the bank's retail loans are...",AgenciesThe bank presently has 20 million acti...,1
2,2022-05-27,Karnataka attracted investment commitment of R...,Karnataka is at the forefront in attracting in...,PTIKarnataka Chief Minister Basavaraj Bommai.K...,1
3,2022-04-06,Splitting of provident fund accounts may be de...,The EPFO is likely to split accounts only at t...,Getty ImagesThe budget for FY22 had imposed in...,0
4,2022-06-14,Irdai weighs proposal to privatise Insurance I...,"Set up in 2009 as an advisory body, IIB collec...",AgenciesThere is a view in the insurance indus...,1
...,...,...,...,...,...
395,2022-06-10,"Banks take a cue from RBI, hike lending rates",These banks raised their respective external b...,"PTIICICI Bank, Bank of Baroda, Punjab National...",0
396,2022-06-29,Sebi issues Rs 27 lakh recovery notice to indi...,"In the event of non-payment, it will recover t...",ReutersThe logo of the Securities and Exchange...,0
397,2022-06-06,Apollo Hospital shares drop 0.68% as Sensex ...,"A total of 10,105 shares changed hands on the ...",Getty ImagesShrikant Chouhan of Kotak Securiti...,0
398,2022-05-16,SBI at Rs 710? What makes analysts see up to 5...,Calling the stock 'attractively valued' analys...,AgenciesThe PSU bank reported a 41.27 per cent...,1


In [ ]:
df3 = pd.read_csv('/content/Sentiment_dataset.csv')
df3['sentiment'] = df3['sentiment'].replace({0.0: 0, 1.0: 1})

# Change the data type of the 'sentiment' column to int64
df3['sentiment'] = df3['sentiment'].astype('int64')

# Print the DataFrame to see the changes
df3

,news_title,reddit_title,sentiment,text,url
0,Mark Cuban launches generic drug company,Billionaire Mark Cuban just launched a drug co...,1,Billionaire investor and Shark Tank star Mark ...,https://www.beckershospitalreview.com/pharmacy...
1,From Defendant to Defender: One Wrongfully Con...,"Man falsely imprisoned for 10 years, uses pris...",1,Attorney Jarrett Adams recently helped overtur...,https://www.nbcnews.com/news/us-news/defendant...
2,"Amazon Tribe Wins Lawsuit Against Big Oil, Sav...",Amazon tribe wins legal battle against oil com...,1,The Amazon Rainforest is well known across the...,https://www.disclose.tv/amazon-tribe-wins-laws...
3,Newark police: No officer fired a single shot ...,Newark police: No officer fired a single shot ...,1,Newark police: No officer fired a single shot ...,https://newjersey.news12.com/newark-police-no-...
4,Ingen barn døde i trafikken i 2019,No children died in traffic accidents in Norwa...,1,I 1970 døde det 560 mennesker i den norske tra...,https://www.nrk.no/trondelag/ingen-barn-dode-i...
...,...,...,...,...,...
843,Dee Why attack: Man allegedly choked and threa...,Dee Why attack: Man allegedly choked and threa...,0,Frightening details have emerged about a toile...,https://www.9news.com.au/2018/11/30/17/55/sydn...
844,Africa: Children and HIV/Aids - 'We Need to Ta...,Africa: Children and HIV/Aids - 'We Need to Ta...,0,"interview\n\nJohannesburg — 360,000 adolescent...",https://allafrica.com/stories/201811300567.html
845,Terrorism suspected in Eilat attack,Terrorism suspected in Eilat attack,0,A violent attack in the southern Israeli port ...,http://www.israelnationalnews.com/News/News.as...
846,Anti-Semitism never disappeared in Europe. It'...,Anti-Semitism never disappeared in Europe. It'...,0,"It's a 17-year-old boy, too frightened to wear...",https://edition.cnn.com/2018/11/27/europe/anti...


In [ ]:
df3.rename(columns={'news_title': 'Headline', 'sentiment': 'label'}, inplace=True)
df3.to_csv('full_articles2.csv', index=False)

In [ ]:
df4 = pd.read_csv('/content/full_articles2.csv')
df4

,Headline,reddit_title,label,text,url
0,Mark Cuban launches generic drug company,Billionaire Mark Cuban just launched a drug co...,1,Billionaire investor and Shark Tank star Mark ...,https://www.beckershospitalreview.com/pharmacy...
1,From Defendant to Defender: One Wrongfully Con...,"Man falsely imprisoned for 10 years, uses pris...",1,Attorney Jarrett Adams recently helped overtur...,https://www.nbcnews.com/news/us-news/defendant...
2,"Amazon Tribe Wins Lawsuit Against Big Oil, Sav...",Amazon tribe wins legal battle against oil com...,1,The Amazon Rainforest is well known across the...,https://www.disclose.tv/amazon-tribe-wins-laws...
3,Newark police: No officer fired a single shot ...,Newark police: No officer fired a single shot ...,1,Newark police: No officer fired a single shot ...,https://newjersey.news12.com/newark-police-no-...
4,Ingen barn døde i trafikken i 2019,No children died in traffic accidents in Norwa...,1,I 1970 døde det 560 mennesker i den norske tra...,https://www.nrk.no/trondelag/ingen-barn-dode-i...
...,...,...,...,...,...
843,Dee Why attack: Man allegedly choked and threa...,Dee Why attack: Man allegedly choked and threa...,0,Frightening details have emerged about a toile...,https://www.9news.com.au/2018/11/30/17/55/sydn...
844,Africa: Children and HIV/Aids - 'We Need to Ta...,Africa: Children and HIV/Aids - 'We Need to Ta...,0,"interview\n\nJohannesburg — 360,000 adolescent...",https://allafrica.com/stories/201811300567.html
845,Terrorism suspected in Eilat attack,Terrorism suspected in Eilat attack,0,A violent attack in the southern Israeli port ...,http://www.israelnationalnews.com/News/News.as...
846,Anti-Semitism never disappeared in Europe. It'...,Anti-Semitism never disappeared in Europe. It'...,0,"It's a 17-year-old boy, too frightened to wear...",https://edition.cnn.com/2018/11/27/europe/anti...


In [ ]:
full_articles_merged = pd.concat([df2[['Headline', 'text', 'label']], df4[['Headline', 'text', 'label']]], ignore_index=True)

# Save the merged DataFrame to CSV
full_articles_merged.to_csv('full_articles_merged.csv', index=False)

,Headline,text,label
0,"Banks holding on to subsidy share, say payment...",ReutersPayments companies and banks are at log...,0
1,Digitally ready Bank of Baroda aims to click o...,AgenciesThe bank presently has 20 million acti...,1
2,Karnataka attracted investment commitment of R...,PTIKarnataka Chief Minister Basavaraj Bommai.K...,1
3,Splitting of provident fund accounts may be de...,Getty ImagesThe budget for FY22 had imposed in...,0
4,Irdai weighs proposal to privatise Insurance I...,AgenciesThere is a view in the insurance indus...,1
...,...,...,...
1243,Dee Why attack: Man allegedly choked and threa...,Frightening details have emerged about a toile...,0
1244,Africa: Children and HIV/Aids - 'We Need to Ta...,"interview\n\nJohannesburg — 360,000 adolescent...",0
1245,Terrorism suspected in Eilat attack,A violent attack in the southern Israeli port ...,0
1246,Anti-Semitism never disappeared in Europe. It'...,"It's a 17-year-old boy, too frightened to wear...",0


In [ ]:
df5 = preprocessing('/content/full_articles_merged.csv')
df5

,Headline,text,label,tokens
0,"Banks holding on to subsidy share, say payment...",ReutersPayments companies and banks are at log...,0,"[reuterspayments, company, bank, loggerhead, s..."
1,Digitally ready Bank of Baroda aims to click o...,AgenciesThe bank presently has 20 million acti...,1,"[agenciesthe, bank, presently, million, active..."
2,Karnataka attracted investment commitment of R...,PTIKarnataka Chief Minister Basavaraj Bommai.K...,1,"[ptikarnataka, chief, minister, basavaraj, chi..."
3,Splitting of provident fund accounts may be de...,Getty ImagesThe budget for FY22 had imposed in...,0,"[getty, imagesthe, budget, imposed, interest, ..."
4,Irdai weighs proposal to privatise Insurance I...,AgenciesThere is a view in the insurance indus...,1,"[agenciesthere, view, insurance, industry, iib..."
...,...,...,...,...
1243,Dee Why attack: Man allegedly choked and threa...,Frightening details have emerged about a toile...,0,"[frightening, detail, emerged, toilet, block, ..."
1244,Africa: Children and HIV/Aids - 'We Need to Ta...,"interview\n\nJohannesburg — 360,000 adolescent...",0,"[interview, johannesburg, adolescent, projecte..."
1245,Terrorism suspected in Eilat attack,A violent attack in the southern Israeli port ...,0,"[violent, attack, southern, israeli, port, cit..."
1246,Anti-Semitism never disappeared in Europe. It'...,"It's a 17-year-old boy, too frightened to wear...",0,"[boy, frightened, wear, kippa, religious, skul..."


In [ ]:
# Convert label column to numeric format in df5
df5['label'] = pd.to_numeric(df5['label'], errors='coerce')  # 'coerce' to handle non-numeric values

# Drop rows with NaN values in label column
df5.dropna(subset=['label'], inplace=True)

# Convert label column to integer type
df5['label'] = df5['label'].astype(int)

# Convert text to sequences using the tokenizer trained on the original dataset
sequences_new = tokenizer.texts_to_sequences(df5['tokens'])

# Pad sequences
padded_sequences_new = pad_sequences(sequences_new, maxlen=maxlen, padding='post', truncating='post')

# Prepare labels
sentiment_labels_new = df5['label'].values

# Split data into train and test sets
x_train_new, x_test_new, y_train_new, y_test_new = train_test_split(padded_sequences_new, sentiment_labels_new, test_size=0.2)



In [ ]:
from tensorflow.keras.optimizers import Adam

# Define the optimizer with the desired learning rate
optimizer = Adam(learning_rate=0.0001)  # Adjust the learning rate as needed

# Compile the model with the defined optimizer
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with early stopping
history = model.fit(x_train_new, y_train_new, batch_size=64, epochs=100, validation_split=0.1, callbacks=[early_stopping])

# Evaluate the model on the test set
loss, accuracy = model.evaluate(x_test_new, y_test_new)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

Epoch 1/100
15/15 [==============================] - 4s 133ms/step - loss: 0.1540 - accuracy: 0.9476 - val_loss: 0.3782 - val_accuracy: 0.8700
Epoch 2/100
15/15 [==============================] - 2s 108ms/step - loss: 0.1464 - accuracy: 0.9454 - val_loss: 0.3791 - val_accuracy: 0.8700
Epoch 3/100
15/15 [==============================] - 2s 163ms/step - loss: 0.1432 - accuracy: 0.9521 - val_loss: 0.3825 - val_accuracy: 0.8600
Epoch 4/100
15/15 [==============================] - 3s 168ms/step - loss: 0.1360 - accuracy: 0.9599 - val_loss: 0.3765 - val_accuracy: 0.8500
Epoch 5/100
15/15 [==============================] - 2s 122ms/step - loss: 0.1269 - accuracy: 0.9621 - val_loss: 0.3699 - val_accuracy: 0.8600
Epoch 6/100
15/15 [==============================] - 2s 126ms/step - loss: 0.1211 - accuracy: 0.9599 - val_loss: 0.3710 - val_accuracy: 0.8600
Epoch 7/100
15/15 [==============================] - 2s 100ms/step - loss: 0.1215 - accuracy: 0.9576 - val_loss: 0.3736 - val_accuracy: 0.8600

In [ ]:
# Save the model
model.save('sentiment_analysis_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from bs4 import BeautifulSoup
import requests
import csv

topic = input("Enter name : ")
url = f'https://news.google.com/search?q={topic}&hl=en-IN&gl=IN&ceid=IN%3Aen'
# url = 'https://news.google.com/home?hl=en-IN&gl=IN&ceid=IN:en'
page = requests.get(url)

soup = BeautifulSoup(page.text, 'html.parser')
articles = soup.find_all('a', class_='WwrzSb')
articles_list = []

for article in articles:
    link = 'https://news.google.com' + article['href'][1:]
    try:
        p1 = requests.get(link)
        if p1 is None:
            continue  # Skip this article if unable to retrieve the content

        if p1.status_code == 403:  # Access Denied
            continue

        if p1.status_code == 404:  # Not Found
            continue

        soup1 = BeautifulSoup(p1.text, 'html.parser')
        heading_tag = soup1.find('h1')

        if heading_tag:
            heading = heading_tag.get_text(strip=True)
            text = '\n'.join([p.get_text(strip=True) for p in soup1.find_all('p')])
            articles_list.append((heading, text))
    except requests.exceptions.ConnectionError as e:
        print(f"Connection error occurred: {e}")
        continue  # Continue to the next iteration if connection error occurs

# Save the scraped articles in a CSV file
file_name = f"{topic}_articles.csv"
with open(file_name, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Heading', 'text'])
    writer.writerows(articles_list)

print("Articles saved in:", file_name)


Enter name : google
Connection error occurred: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Connection error occurred: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Connection error occurred: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Articles saved in: google_articles.csv


In [ ]:
news = preprocessing('/content/google_articles.csv')
news

,Heading,text,tokens
0,Google plans to charge for AI-powered search e...,To enjoy additional benefits\nCONNECT WITH US\...,"[enjoy, additional, benefit, connect, u, april..."
1,Google set to charge for internet searches wit...,Cost of artificial intelligence service could ...,"[cost, artificial, intelligence, service, coul..."
2,Google is planning to charge you for these fea...,The TOI Tech Desk is a dedicated team of journ...,"[toi, tech, desk, dedicated, team, journalist,..."
3,Google parent Alphabet weighs offer for HubSpot,The potential deal would also allow Alphabet t...,"[potential, deal, would, also, allow, alphabet..."
4,Google parent Alphabet weighs offer for HubSpot,5 Stories\n7 Stories\n9 Stories\n9 Stories\n8 ...,"[story, story, story, story, story, story, pri..."
...,...,...,...
85,Google to Delete Billions of Browsing Records ...,Google has agreed to purge billions of data re...,"[google, agreed, purge, billion, data, record,..."
86,Google to delete some data it collected on ‘pr...,Google agreed to delete data points it collect...,"[google, agreed, delete, data, point, collecte..."
87,Google could put search results behind paywall...,Notifications can be managed in browser prefer...,"[notification, managed, browser, preference, p..."
88,Google just scored a big win in the AI talent war,\n\n\n\n\n\n\nGoogle has a new feather in its ...,"[google, new, feather, cap, luring, openai, fo..."


In [ ]:
from tensorflow.keras.models import load_model

# Load the model
loaded_model = load_model('sentiment_analysis_model.h5')
sequences = tokenizer.texts_to_sequences(news)

# Pad sequences
padded_sequences = pad_sequences(sequences, maxlen=maxlen, padding='post', truncating='post')

# Predict sentiments
predictions = loaded_model.predict(padded_sequences)

# Filter out articles with negative sentiment
negative_articles = []
for i, prediction in enumerate(predictions):
    if prediction < 0.5:  # Assuming 0.5 is the threshold for negative sentiment
        negative_articles.append(news.iloc[i]['Heading'])

# Return headings for negative articles
for heading in negative_articles:
    print(heading)

1/1 [==============================] - 0s 86ms/step
Google plans to charge for AI-powered search engine: Report
Google set to charge for internet searches with AI, reports say
Google is planning to charge you for these features while Googling
Google parent Alphabet weighs offer for HubSpot


In [ ]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Function for preprocessing
def preprocessing(text):
    # Tokenization
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.lower() not in stop_words]

    # Convert tokens to lowercase
    tokens = [token.lower() for token in tokens]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return ' '.join(tokens)

def get_negative_articles(topic):
    # Scrape news articles
    url = f'https://news.google.com/search?q={topic}&hl=en-IN&gl=IN&ceid=IN%3Aen'
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    articles = soup.find_all('a', class_='WwrzSb')
    articles_list = []

    for article in articles:
        link = 'https://news.google.com' + article['href'][1:]
        try:
            p1 = requests.get(link)
            if p1 is None or p1.status_code in [403, 404]:
                continue

            soup1 = BeautifulSoup(p1.text, 'html.parser')
            heading_tag = soup1.find('h1')

            if heading_tag:
                heading = heading_tag.get_text(strip=True)
                text = '\n'.join([p.get_text(strip=True) for p in soup1.find_all('p')])
                articles_list.append((heading, text))
        except requests.exceptions.ConnectionError as e:
            print(f"Connection error occurred: {e}")
            continue

    # Save scraped articles in a CSV file
    file_name = f"{topic}_articles.csv"
    with open(file_name, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Heading', 'Text'])
        writer.writerows(articles_list)

    # Preprocess scraped articles
    news = pd.DataFrame(articles_list, columns=['Heading', 'Text'])
    news['Text'] = news['Text'].apply(preprocessing)

    # Drop rows with NaN in the 'Text' column
    news.dropna(subset=['Text'], inplace=True)

    # Load sentiment analysis model and tokenizer
    loaded_model = load_model('sentiment_analysis_model.h5')
    tokenizer = Tokenizer(num_words=5000)
    tokenizer.fit_on_texts(news['Text'])

    # Pad sequences
    sequences = tokenizer.texts_to_sequences(news['Text'])
    maxlen = 100
    padded_sequences = pad_sequences(sequences, maxlen=maxlen, padding='post', truncating='post')

    # Predict sentiments
    predictions = loaded_model.predict(padded_sequences)

    # Filter out articles with negative sentiment
    negative_articles = news[predictions < 0.5]

    return negative_articles

# Example usage
topic = input("Enter the topic: ")
negative_articles = get_negative_articles(topic)
print("Negative Articles:")
print(negative_articles[['Heading', 'Text']])



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Enter the topic: microsoft
Connection error occurred: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Connection error occurred: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
2/2 [==============================] - 0s 14ms/step
Negative Articles:
                                              Heading  \
2   US Blames Microsoft 'Cascade of Errors' For Ch...   
13  IBM and Microsoft Collaborate to Launch Experi...   
36                                      Access Denied   
47                         Microsoft customer stories   
51  How to Get the New Copilot Features for Micros...   

                                                 Text  
2   published : pragati pal afp last updated : apr...  
13  new experience zone enable client explore join...  
36                                                     
47     see microsoft tool help company run business .  
51  roll-out ai feature popular platfor

In [ ]:
file_name = f"{topic}_negative_articles.csv"
negative_articles.to_csv(file_name, index=False)
print("Negative articles saved in:", file_name)

Negative articles saved in: microsoft_negative_articles.csv


In [ ]:
df = pd.read_csv(f'/content/{topic}_negative_articles.csv')
df

,Heading,Text
0,US Blames Microsoft 'Cascade of Errors' For Ch...,published : pragati pal afp last updated : apr...
1,IBM and Microsoft Collaborate to Launch Experi...,new experience zone enable client explore join...
2,Access Denied,NaN
3,Microsoft customer stories,see microsoft tool help company run business .
4,How to Get the New Copilot Features for Micros...,roll-out ai feature popular platform continues...
